In [1]:
# Importação das bibliotecas
import pyvis as pv
import networkx as nx
from pyvis.network import Network
import plotly.graph_objects as go
import dash
from dash import dcc
from dash import html
import dash_cytoscape as cyto
from dash.dependencies import Input, Output
import plotly.express as px
import pandas as pd
import random as rd
import numpy as np
from matplotlib.pyplot import figure

In [2]:
# Importando as áreas
raw_data = pd.read_csv("c:/Users/Usuario/Documents/Estudos/Data Science e Programação/Datasets/colaboradores.csv")
raw_data = raw_data[['nome', 'area']]

In [3]:
# Criando uma lista de quantidade de pessoas por área
areas = set(raw_data['area'])

In [4]:
# Criando uma função para encontrar a estimativa do número de conecções 
def total_conections(name):
    area = list(raw_data.loc[raw_data['nome'] == name]['area'])[0]
    total = len(raw_data.loc[raw_data['area'] == area])
    conections = rd.randint(round(total*0.1), round(total*1.5))
    return conections

In [5]:
# Criando uma função para atribuir a ponderação de cada nome
def weight_definition(name):
    # Encontrando a área
    areas_dict = dict.fromkeys(areas, 1)
    area = list(raw_data.loc[raw_data['nome'] == name]['area'])[0]
    # Atribuindo o peso adequado
    areas_dict[area] = 100
    raw_data['probabilidade'] = raw_data.apply(lambda x: areas_dict[x['area']], axis = 1)
    # Criando o vetor de itens e pesos
    raw_data_filter = raw_data.loc[raw_data['nome'] != name]
    itens = list(raw_data_filter['nome'])
    probability = list(raw_data_filter['probabilidade'])
    weights = [j/sum(probability) for j in probability]
    return itens, weights

In [6]:
# Criando uma função para criar a lista de conecções de cada pessoa
def ona_conections(name):
    # Total de conecções
    conections = total_conections(name)
    # Definição da lista de nomes e pesos
    names_list = weight_definition(name)[0]
    weights_list = weight_definition(name)[1]
    # Criação da lista de coneções
    conections_list = np.random.choice(a = names_list, size = conections, p = weights_list)
    return conections_list  

In [7]:
# Criando uma função para gerar a rede de cada nome
def ona_generator(name):
    conections_list = ona_conections(name)
    ona_list = [(name, i) for i in conections_list]
    return ona_list

In [8]:
# Construindo a rede organizacional
ona_total_list = []
for i in list(raw_data['nome']):
    ona_total_list.extend(ona_generator(i))

In [9]:
# Criação do dataset
dataset = pd.DataFrame(ona_total_list, columns = ['partida', 'chegada'])

In [24]:
# Criação dos vetores
source = list(dataset['partida'])
source_size = [source.count(i) for i in source]
target = list(dataset['chegada'])
target_size = [target.count(i) for i in target]
edge = list(zip(source, source_size, target, target_size))

In [26]:
graph = Network(height='750px', width='100%', bgcolor='#111111', font_color='white', directed = True)
graph.barnes_hut()

In [27]:
for e in edge:
    src = e[0]
    src_size = e[1]
    dst = e[2]
    dst_size = e[3]

    graph.add_node(src, src, title = src, size = src_size)
    graph.add_node(dst, dst, title = dst, size = dst_size)
    graph.add_edge(src, dst)

In [28]:
graph.show('graph.html')